# Donnée de l'INPI

La class `inpi` a des erreurs. Ne pas tenir compte de ce notebook

Dans les données de stock, il y a un stock inital a date du 5 mai 2017 et un ensemble de stock initial délivré à différent intervals.

Il y a aussi un ensemble de données provenant de flux. -> données de flux

Le format de donnée est stocké dans le json suivant:

- https://s3.console.aws.amazon.com/s3/object/calfdata/INPI/TC_1/Stock/dtypes_stock.json

## Documentation

- [Doc INSEE](https://scm.saas.cagip.group.gca/PERNETTH/database/blob/master/Documentation/IMR/Doc_Tech_IMR_Mai_2019_v1.5.1.pdf)
- [etalab](https://github.com/etalab/rncs_worker_api_entreprise)
    - [Module data cleaning](https://github.com/etalab/rncs_worker_api_entreprise/blob/master/clean_csv.bash)
- Path local: `C:\Users\PERNETTH\Documents\Projects\GitLab\CreationDataBase\INPI\TC_1\Stock\Stock_initial`
- Path Reseau: `G:\Projets\PP2019\2019_PROGRAMME BATICA\DATA\INPI\sources\IMR_Donnees_Saisies\tc\stock\2017`

### Variables communes

![image](https://drive.google.com/uc?export=view&id=1laCGFEtxM4bbckNZQEkw1icBjF4pd20-)

### Stock

Le contenu de chaque fichier est disponible [ici](https://scm.saas.cagip.group.gca/PERNETTH/database/tree/master/INPI/TC_1)

Les données brutes sont répertoriées dans des fichiers zip composées de 7 fichiers:

- Actes
- Comptes Annuels
- ETS
- OBS
- PM
- PP
- REP

### Donnees de flux 

- Path local: `C:\Users\PERNETTH\Documents\Projects\GitLab\CreationDataBase\INPI\TC_1\Flux`
- Path Reseau: `G:\Projets\PP2019\2019_PROGRAMME BATICA\DATA\INPI\sources\IMR_Donnees_Saisies\tc\Flux`

Les PP en local sont disponibles ici: `C:\Users\PERNETTH\Documents\Projects\GitLab\CreationDataBase\INPI\TC_1\Flux\PP`

name csv : `0101_1_20170512_112544_4_PP_EVT`
    
    - code greffe: 0101
    - numéro transmission: 1
    - Date: 2017-05-12
    - heures: 11h25m44s
    - Type: PP
    - Evenement: Oui

**FICHIERS TRANSMIS EN CAS DE MISE A JOUR D’UN DOSSIER (EVENEMENT)**

En cas de mise à jour d’un dossier suite à un événement (modification, radiation), les fichiers transmis ont une structure identique aux fichiers créés à l’immatriculation avec la présence de 2 champs spécifiques : la date de l’événement (Date_Greffe) et le libellé de l’événement (Libelle_Evt).

Dans ces cas, 6 types de fichiers supplémentaires, numérotés, sont transmis correspondant à :
* Evénements modifiant ou complétant les dossiers d’immatriculation des personnes morales (2) ou physiques (4)
* Evénements modifiant ou complétant les informations relatives aux représentants (6) ou aux établissements (9)
* Evénements supprimant des représentants (7 – Représentant partant) ou des établissements (10 – Etablissement supprimé)

![](https://drive.google.com/uc?export=view&id=1FVEGNqogl1NxB84BtdF4TQztCXjmtpyo)

Attention, il peut arriver que le même dossier fasse l’objet de plusieurs événements (création et modification) dans la même transmission. Il est impératif d’intégrer les événements dans l’ordre d’apparition.

Les csv sont classés de la manière suivante:

- Module (ETS, PP, PM, etc)
    - NEW: New event
    - EVT: Update event
    
## REP

Doc REP: https://scm.saas.cagip.group.gca/PERNETTH/database/blob/master/Documentation/IMR/Doc_Tech_IMR_Mai_2019_v1.5.1.pdf
- page 30

Attention, certains champs sont remplis selon le type de la société. Ex SIREN -> Si type =PM

Pourquoi deux SIREN?

Le deuxième siren correspond a celui de la PM qui detient l'entreprise.

Ex: Peugeot a plusieurs détenteur. Le détenteur possède un siren, il sera affiché dans siren.1 uniquement si détenteur est une PM

In [ ]:
#!pip install tqdm

In [ ]:
import boto3, json
import dask.dataframe as dd
import pandas as pd
import inpi as inpi
#from tqdm.notebook import tqdm
import tqdm
%load_ext autoreload
%autoreload 2

In [ ]:
#from sagemaker import get_execution_role
#role = get_execution_role() 
instance_aws = 'https://calfdata.s3.eu-west-3.amazonaws.com'
bucket = 'calfdata'
agg_datasock = inpi.inpiStockFlux(instance_aws, bucket)
agg_datasock.uploadFileBucket(pathfile = 'INPI/TC_1/dtypes_stock.json')
with open('dtypes_stock.json', 'r' ,encoding='utf8') as f:
    datastore = json.load(f)

## Batch Stock aggregation csv

In [ ]:
for key, value in tqdm.tqdm(datastore['option'].items()): 
    if key not in ['PP', 'PM', 'ETS']:
        print(key)
        dtype = value["dtype"]
        parse_dates = value['parse_dates']
        
        for p in [True, False]:
            agg_datasock.appendInpiStock(
                key,
                dtype,
                parse_dates,
                partiel = p,
                return_frame = False)

## Batch flux aggregation csv

### test

La class `inpi` a des erreurs. Ne pas tenir compte de ce notebook

In [ ]:
import glob

def appendInpiFlux(
                       option_extract,
                       dtype,
                       parse_date,
                       new = True,
                       year = 2017,
                       return_frame = True,
                       to_s3 = True):

        """
        Append all csv files in a folder to a Pandas DataFrame and save the
        output in S3: INPI/TC_1/Stock_processed from S3 bucket:
        /INPI/TC_1/Flux/


        format output-> gz
        - Option + filename +stock +gz:
            - 2017_NEW_PP.gz

        Args:
        option_extract: String, from list ['ACTES', 'COMPTES_ANNUELS','ETS',
                      'OBS', 'PM', 'PP','REP']
        dtype: variables type, use 'str', for the data and pd.Int64Dtype()
        for integer. If possible
        parse_date: A list with the variables to convert into dates
        new: Boolean, if true, then go NEW folder, else EVT
        year: integer. Locate year folder, 2017,2018,2019,2020

        Return:
            Pandas DataFrame
        """

        # Test if in
        list_option = ["ACTES", "COMPTES_ANNUELS", "ETS",
                      "OBS", "PM", "PP", "REP"]

        if option_extract not in list_option:
            return print(
            "Veuillez utiliser l'un des mots clés suivants {} \n \
        pour l'argument origin".format(list_option)
        )

        #my_bucket = self.s3.Bucket(self.bucket)

        list_mois =  [#"01", "02", "03", "04",
                      "05",
            #"06","07","08", "09", "10", "11", "12"
        ]
        path_flux = r"C:\Users\PERNETTH\Documents\Projects\GitLab" \
        r"\CreationDataBase\INPI\TC_1\Flux"
        list_errors = []
        df_append = pd.DataFrame()
        for month in list_mois:
            if new:
                subfilter = r"{0}\{1}\{2}\NEW".format(str(year),
                                                      month,
                                                      option_extract)
                new_option = 'NEW'
            else:
                subfilter = r"{0}\{1}\{2}\EVT".format(str(year),
                                                      month,
                                                      option_extract)
                new_option = 'EVT'
            #print(r"{}\{}".format(path_flux, subfilter))
            
            import os
            for filename in os.listdir(r"{}\{}".format(path_flux, subfilter)):
                f = r"{}\{}\{}".format(path_flux, subfilter, filename)
                try:
                    df_ = pd.read_csv(f,
                            sep=";", 
                            dtype=datastore['option']['ETS']['dtype'],
                            parse_dates=
                            datastore['option']['ETS']['parse_dates'],
                                      error_bad_lines = False
                                     )
                    df_append = df_append.append(df_, sort = False)
                except:
                    list_errors.append(f)
            
                #df_ = pd.concat(
        #[
         #   pd.read_csv(file, sep=";", 
         #               dtype=dtype, 
         #               parse_dates=parse_date)
        #],
        #"ignore_index=True,
    #)
                
        return df_append, list_errors

In [ ]:
temp = appendInpiFlux(option_extract="ETS",
                            new=False,
                            year=2017,
                            dtype=datastore['option']['ETS']['dtype'],
                            parse_date=datastore['option']['ETS']['parse_dates'],
                            return_frame=True,
                            to_s3 = False)
#C:\Users\PERNETTH\Documents\Projects\GitLab\CreationDataBase\INPI\TC_1\Flux\2017\05\ETS\EVT

### Batch

In [ ]:
list_failed = []
for key, value in tqdm.tqdm(datastore['option'].items()): 
    if key not in ['OBS']:
        print(key)
        try:
            dtype = value["dtype"]
            parse_dates = value['parse_dates']
            agg_datasock.appendInpiFlux(
            option_extract =key,
            new = True,
            year = 2017,
            dtype = dtype,
            parse_date = parse_dates,
            return_frame = True)
        except:
            list_failed.append(key)

In [ ]:
list_failed = []
for key, value in tqdm.tqdm(datastore['option'].items()): 
    if key not in ['PM', 'PP']:
        print(key)
        try:
            dtype = value["dtype"]
            parse_dates = value['parse_dates']
            agg_datasock.appendInpiFlux(
            option_extract =key,
            new = False,
            year = 2017,
            dtype = dtype,
            parse_date = parse_dates,
            return_frame = True)
        except:
            list_failed.append(key)

In [ ]:
import os
os.remove('dtypes_stock.json')

In [ ]:
#agg_datasock.appendInpiStock('PP',
#                             dtype,
#                             parse_dates)

In [ ]:
#agg_datasock.appendInpiStock('PP',
#                                    dtype,
#                                    parse_dates,
#                                    partiel = True)